# Applied Data Finance ML Models - Model Registry

This notebook trains the three ML models required for the Applied Data Finance Intelligence Agent:
- **Payment Volume Forecasting** - Predict future monthly servicing cash-flow
- **Borrower Risk Classification** - Identify borrowers likely to become delinquent
- **Collection Promise-to-Pay Prediction** - Estimate the probability that a collection event results in a valid promise

All models are registered to Snowflake Model Registry and exposed to the agent through `sql/ml/07_create_model_wrapper_functions.sql`.

## Prerequisites

**Required Packages** (configured automatically):
- `snowflake-ml-python`
- `scikit-learn`
- `xgboost`
- `matplotlib`

**Database Context:**
- **Database:** ADF_INTELLIGENCE  
- **Schema:** ANALYTICS  
- **Warehouse:** ADF_SI_WH

**Note:** This notebook uses Snowflake Model Registry. Ensure you have appropriate permissions to create and register models.


## Import Required Packages


In [ ]:
# Import Python packages
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

# Import Snowpark
from snowflake.snowpark.context import get_active_session
import snowflake.snowpark.functions as F
import snowflake.snowpark.types as T
from snowflake.snowpark import Window

# Import Snowpark ML
from snowflake.ml.modeling.preprocessing import StandardScaler, OneHotEncoder
from snowflake.ml.modeling.pipeline import Pipeline
from snowflake.ml.modeling.linear_model import LinearRegression, LogisticRegression
from snowflake.ml.modeling.ensemble import RandomForestClassifier
from snowflake.ml.modeling.metrics import mean_squared_error, mean_absolute_error, accuracy_score, roc_auc_score
from snowflake.ml.registry import Registry

print("✅ Packages imported successfully")


## Connect to Snowflake

Get the active session and set context to the Applied Data Finance database.


In [ ]:
# Get active Snowflake session
session = get_active_session()

# Set context
session.use_database('ADF_INTELLIGENCE')
session.use_schema('ANALYTICS')
session.use_warehouse('ADF_SI_WH')

print(f"✅ Connected - Role: {session.get_current_role()}")
print(f"   Warehouse: {session.get_current_warehouse()}")
print(f"   Database.Schema: {session.get_fully_qualified_current_schema()}")


---
# MODEL 1: Payment Volume Forecasting

Predict future monthly cash collections using historical payment activity.


### Prepare Revenue Training Data


In [ ]:
# Get monthly payment volume data with servicing features
payment_volume_df = session.sql("""
SELECT
    DATE_TRUNC('month', payment_date)::DATE AS payment_month,
    MONTH(payment_date) AS month_num,
    YEAR(payment_date) AS year_num,
    SUM(amount)::FLOAT AS total_payment_amount,
    COUNT(DISTINCT loan_id)::FLOAT AS loan_count,
    AVG(amount)::FLOAT AS avg_payment_amount,
    COUNT_IF(late_fee_applied)::FLOAT AS late_payment_count
FROM RAW.PAYMENT_HISTORY
WHERE payment_date >= DATEADD('month', -36, CURRENT_DATE())
GROUP BY 1,2,3
ORDER BY payment_month
""")

print(f"Payment volume data: {payment_volume_df.count()} months")
payment_volume_df.show(5)


### Split Data and Train Revenue Model


In [ ]:
# Train/test split (last 6 months for testing)
train_payment_volume = payment_volume_df.filter(F.col("PAYMENT_MONTH") < F.dateadd("month", F.lit(-6), F.current_date()))
test_payment_volume = payment_volume_df.filter(F.col("PAYMENT_MONTH") >= F.dateadd("month", F.lit(-6), F.current_date()))

# Drop PAYMENT_MONTH (DATE type not supported in pipeline)
train_payment_volume = train_payment_volume.drop("PAYMENT_MONTH")
test_payment_volume = test_payment_volume.drop("PAYMENT_MONTH")

# Create pipeline
payment_volume_pipeline = Pipeline([
    ("Scaler", StandardScaler(
        input_cols=["MONTH_NUM", "LOAN_COUNT", "AVG_PAYMENT_AMOUNT", "LATE_PAYMENT_COUNT"],
        output_cols=["MONTH_NUM_SCALED", "LOAN_COUNT_SCALED", "AVG_PAYMENT_AMOUNT_SCALED", "LATE_PAYMENT_COUNT_SCALED"]
    )),
    ("LinearRegression", LinearRegression(
        label_cols=["TOTAL_PAYMENT_AMOUNT"],
        output_cols=["PREDICTED_PAYMENT_AMOUNT"]
    ))
])

# Train model
payment_volume_pipeline.fit(train_payment_volume)
print("✅ Payment volume forecasting model trained")


### Evaluate and Register Revenue Model


In [ ]:
# Make predictions on test set
test_predictions = payment_volume_pipeline.predict(test_payment_volume)

# Calculate metrics
mae = mean_absolute_error(df=test_predictions, y_true_col_names="TOTAL_PAYMENT_AMOUNT", y_pred_col_names="PREDICTED_PAYMENT_AMOUNT")
mse = mean_squared_error(df=test_predictions, y_true_col_names="TOTAL_PAYMENT_AMOUNT", y_pred_col_names="PREDICTED_PAYMENT_AMOUNT")
rmse = mse ** 0.5

metrics = {"mae": round(mae, 2), "rmse": round(rmse, 2)}
print(f"Model metrics: {metrics}")

# Register model
reg = Registry(session)
reg.log_model(
    model=payment_volume_pipeline,
    model_name="PAYMENT_VOLUME_FORECASTER",
    version_name="V1",
    comment="Predicts monthly payment volume based on servicing activity using Linear Regression",
    metrics=metrics
)

print("✅ Payment volume model registered to Model Registry as PAYMENT_VOLUME_FORECASTER")


---
# MODEL 2: Borrower Risk Classification

Classify borrowers as likely to become delinquent based on recent servicing performance.


### Prepare Churn Training Data


In [ ]:
# Get borrower features for delinquency risk classification
borrower_risk_df = session.sql("""
SELECT
    c.customer_id,
    c.risk_segment AS borrower_segment,
    c.employment_status,
    c.annual_income::FLOAT AS annual_income,
    c.credit_score::FLOAT AS credit_score,
    COUNT(DISTINCT l.loan_id)::FLOAT AS total_loans,
    SUM(l.outstanding_principal)::FLOAT AS outstanding_principal,
    COUNT_IF(l.servicing_status = 'DELINQUENT')::FLOAT AS delinquent_loans,
    AVG(p.amount)::FLOAT AS avg_payment_amount,
    COUNT_IF(p.nsf_flag)::FLOAT AS nsf_events,
    (COUNT_IF(l.servicing_status = 'DELINQUENT') > 0)::BOOLEAN AS is_delinquent
FROM RAW.CUSTOMERS c
LEFT JOIN RAW.LOAN_ACCOUNTS l ON c.customer_id = l.customer_id
LEFT JOIN RAW.PAYMENT_HISTORY p ON l.loan_id = p.loan_id
GROUP BY 1,2,3,4,5
HAVING COUNT(DISTINCT l.loan_id) > 0
LIMIT 10000
""")

print(f"Borrower risk data: {borrower_risk_df.count()} borrowers")
borrower_risk_df.show(5)


### Train Churn Classification Model


In [ ]:
# Train/test split (80/20)
train_risk, test_risk = borrower_risk_df.random_split([0.8, 0.2], seed=42)

# Drop CUSTOMER_ID
train_risk = train_risk.drop("CUSTOMER_ID")
test_risk = test_risk.drop("CUSTOMER_ID")

# Create pipeline with preprocessing and classification
borrower_risk_pipeline = Pipeline([
    ("Encoder", OneHotEncoder(
        input_cols=["BORROWER_SEGMENT", "EMPLOYMENT_STATUS"],
        output_cols=["BORROWER_SEGMENT_ENCODED", "EMPLOYMENT_STATUS_ENCODED"],
        drop_input_cols=True,
        handle_unknown="ignore"
    )),
    ("Classifier", RandomForestClassifier(
        label_cols=["IS_DELINQUENT"],
        output_cols=["RISK_PREDICTION"],
        n_estimators=200,
        max_depth=12
    ))
])

# Train model
borrower_risk_pipeline.fit(train_risk)
print("✅ Borrower risk classification model trained")


### Evaluate and Register Churn Model


In [ ]:
# Make predictions
borrower_risk_predictions = borrower_risk_pipeline.predict(test_risk)

# Calculate metrics
accuracy = accuracy_score(df=borrower_risk_predictions, y_true_col_names="IS_DELINQUENT", y_pred_col_names="RISK_PREDICTION")
risk_metrics = {"accuracy": round(accuracy, 4)}
print(f"Borrower risk model metrics: {risk_metrics}")

# Register model
reg.log_model(
    model=borrower_risk_pipeline,
    model_name="BORROWER_RISK_MODEL",
    version_name="V1",
    comment="Predicts borrower delinquency risk using Random Forest on servicing features",
    metrics=risk_metrics
)

print("✅ Borrower risk model registered to Model Registry as BORROWER_RISK_MODEL")


---
# MODEL 3: Collection Promise-to-Pay Prediction

Predict the probability that a collection interaction results in a valid promise to pay.


### Prepare Device Deployment Success Data


In [ ]:
# Get collection interaction features for promise-to-pay prediction
collection_ptp_df = session.sql("""
SELECT
    ce.collection_id,
    ce.loan_id,
    ce.customer_id,
    ce.event_type,
    ce.severity,
    ce.outcome,
    COALESCE(ce.promise_amount, 0)::FLOAT AS promise_amount,
    DATEDIFF('day', ce.event_timestamp::DATE, CURRENT_DATE()) AS event_age_days,
    loans.delinquency_bucket,
    loans.servicing_status,
    loans.outstanding_principal::FLOAT AS outstanding_principal,
    (ce.promise_to_pay_date IS NOT NULL OR ce.outcome = 'PROMISE_TO_PAY')::BOOLEAN AS ptp_success
FROM RAW.COLLECTION_EVENTS ce
JOIN RAW.LOAN_ACCOUNTS loans ON ce.loan_id = loans.loan_id
WHERE ce.event_timestamp >= DATEADD('month', -18, CURRENT_TIMESTAMP())
LIMIT 30000
""")

print(f"Collection interactions: {collection_ptp_df.count()} rows")
collection_ptp_df.show(5)


### Train Conversion Model


In [ ]:
# Split data
train_ptp, test_ptp = collection_ptp_df.random_split([0.8, 0.2], seed=42)

# Drop identifier columns
train_ptp = train_ptp.drop("COLLECTION_ID", "LOAN_ID", "CUSTOMER_ID")
test_ptp = test_ptp.drop("COLLECTION_ID", "LOAN_ID", "CUSTOMER_ID")

# Create pipeline
collection_ptp_pipeline = Pipeline([
    ("Encoder", OneHotEncoder(
        input_cols=["EVENT_TYPE", "SEVERITY", "OUTCOME", "DELINQUENCY_BUCKET", "SERVICING_STATUS"],
        output_cols=["EVENT_TYPE_ENC", "SEVERITY_ENC", "OUTCOME_ENC", "DELINQUENCY_BUCKET_ENC", "SERVICING_STATUS_ENC"],
        drop_input_cols=True,
        handle_unknown="ignore"
    )),
    ("Classifier", LogisticRegression(
        label_cols=["PTP_SUCCESS"],
        output_cols=["PTP_PREDICTION"]
    ))
])

# Train
collection_ptp_pipeline.fit(train_ptp)
print("✅ Collection promise-to-pay model trained")


### Evaluate and Register Conversion Model


In [ ]:
# Predict on test set
ptp_predictions = collection_ptp_pipeline.predict(test_ptp)

# Calculate accuracy
ptp_accuracy = accuracy_score(
    df=ptp_predictions,
    y_true_col_names="PTP_SUCCESS",
    y_pred_col_names="PTP_PREDICTION"
)
ptp_metrics = {"accuracy": round(ptp_accuracy, 4)}
print(f"Promise-to-pay model metrics: {ptp_metrics}")

# Register model
reg.log_model(
    model=collection_ptp_pipeline,
    model_name="COLLECTION_SUCCESS_MODEL",
    version_name="V1",
    comment="Predicts the probability that a collection interaction results in a promise to pay",
    metrics=ptp_metrics
)

print("✅ Collection PTP model registered to Model Registry as COLLECTION_SUCCESS_MODEL")


---
# Verify Models in Registry


In [ ]:
# Show all models in the registry
print("Models in registry:")
reg.show_models()

# Show versions for payment volume model
print("\nPayment Volume Forecaster versions:")
reg.get_model("PAYMENT_VOLUME_FORECASTER").show_versions()

# Show versions for borrower risk model  
print("\nBorrower Risk Model versions:")
reg.get_model("BORROWER_RISK_MODEL").show_versions()

# Show versions for collection success model
print("\nCollection Success Model versions:")
reg.get_model("COLLECTION_SUCCESS_MODEL").show_versions()

print("\n✅ All models registered and ready to add to the Intelligence Agent")


---
# Test Model Inference

Test calling each model to make predictions.


In [ ]:
# Test payment volume forecast on recent data
payment_volume_model = reg.get_model("PAYMENT_VOLUME_FORECASTER").default
recent_payment_volume = payment_volume_df.limit(3).drop("PAYMENT_MONTH")
payment_volume_preds = payment_volume_model.run(recent_payment_volume, function_name="predict")
print("Payment volume predictions:")
payment_volume_preds.select("TOTAL_PAYMENT_AMOUNT", "PREDICTED_PAYMENT_AMOUNT").show()

# Test borrower risk prediction on sample borrowers
borrower_risk_model = reg.get_model("BORROWER_RISK_MODEL").default
sample_borrowers = borrower_risk_df.limit(5).drop("CUSTOMER_ID")
borrower_risk_preds = borrower_risk_model.run(sample_borrowers, function_name="predict")
print("\nBorrower risk predictions:")
borrower_risk_preds.select("IS_DELINQUENT", "RISK_PREDICTION").show()

# Test collection promise-to-pay prediction
collection_success_model = reg.get_model("COLLECTION_SUCCESS_MODEL").default
sample_collections = collection_ptp_df.limit(5).drop("COLLECTION_ID", "LOAN_ID", "CUSTOMER_ID")
collection_success_preds = collection_success_model.run(sample_collections, function_name="predict")
print("\nCollection promise-to-pay predictions:")
collection_success_preds.select("PTP_SUCCESS", "PTP_PREDICTION").show()

print("\n✅ All models tested successfully!")


---
# Next Steps

## Add Models to Intelligence Agent

**Option 1: Using the SQL Script (Easiest)**
Run `sql/agent/08_create_intelligence_agent.sql` which automatically configures all 3 ML models.

**Option 2: Manual Configuration in Snowsight**
1. In Snowsight → AI & ML → Agents → ADF_INTELLIGENCE_AGENT
2. Go to Tools → + Add → Function
3. Add each model wrapper procedure:
   - **PREDICT_PAYMENT_VOLUME** (from `sql/ml/07_create_model_wrapper_functions.sql`)
   - **PREDICT_BORROWER_RISK** (from `sql/ml/07_create_model_wrapper_functions.sql`)
   - **PREDICT_COLLECTION_SUCCESS** (from `sql/ml/07_create_model_wrapper_functions.sql`)

## Example Questions for Agent

- "Predict cash collections for the next 6 months"
- "Which borrowers are at high risk of delinquency?"
- "What is the promise-to-pay conversion probability for this collection event?"
- "Which segments contribute most to upcoming payment volume?"

The models will now be available as tools your agent can use!
